In [ ]:
from getDocument import *
from PdfUtils import *
import json
import time
from sentence_transformers import SentenceTransformer
# import torch

In [ ]:
start = time.perf_counter() # To be used later

defaultPaper = get_paper_and_references_embedding_and_titles_from_query("BERT Rediscovers the Classical NLP Pipeline")

mainPaperSections = get_text(get_paperId_from_query(defaultPaper[1][0]))

In [ ]:
# tupleOfResults = get_paper_and_references_embedding_and_titles_from_query("BERT Rediscovers the Classical NLP Pipeline")

# embeddings = tupleOfResults[0]
# titles = tupleOfResults[1]

# print(titles)

In [ ]:
newCosineSimilarity = torch.nn.CosineSimilarity(dim=0)
# mainPaper = torch.tensor(embeddings[0])
# print(f"Main paper title: {titles[0]}")

In [ ]:
# Create a dictionary for the purpose of not repreating titles
title_embedding_dict = {}

listOfSearchedPapers = [] # For speed purposes, took 4min 20sec before

for embedding, title in zip(defaultPaper[0], defaultPaper[1]):
    if title not in listOfSearchedPapers:
        listOfSearchedPapers.append(title)
        subPaper = get_paper_and_references_embedding_and_titles_from_query(title)
        # First depth, all of the main paper's references
        if title not in title_embedding_dict:
            title_embedding_dict[title] = embedding

        # Second depth, all of the referenced paper's references
        for sub_embedding, sub_title in zip(subPaper[0], subPaper[1]):
            if sub_title not in title_embedding_dict:
                title_embedding_dict[sub_title] = sub_embedding
        

In [ ]:
# A function to return just the first value of the list to be used in list.sort()
def getKey(listOfCosAndTitle):
    return listOfCosAndTitle[0]

In [ ]:
newCosineSimilarity = torch.nn.CosineSimilarity(dim=0)
mainPaperEmbedding = torch.tensor(defaultPaper[0][0])

similarityScoresWithTitle = []
# Get the cos similarity between the main paper and all of the subpapers
for key in title_embedding_dict.keys():
    compared_paper = torch.tensor(title_embedding_dict[key])
    similarityScoresWithTitle.append([newCosineSimilarity(mainPaperEmbedding, compared_paper), key])

In [ ]:
GREEN = '\033[32m'
RED = '\033[31m'
RESET = '\033[0m'

similarityScoresWithTitle.sort(reverse=True, key=getKey)
for i in similarityScoresWithTitle:
    print(f"Cos similarity score: {i[0]:.3f} and it's {RED + 'not cited in ' if i[1] not in defaultPaper[1] else GREEN + 'cited in     '}{RESET}the main paper with title: {i[1]:20s}")
print(similarityScoresWithTitle)

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = {}
# for i in listOfSections:
#     embeddings.append(model.encode(i))
# print(embeddings)

embeddings is a dictionary where each key is a paper title, the value is a list of lists
each sublist is -> Section Embedding, Index into the section, the section text, the title again(?)

In [ ]:
#Try parsing the top 10 things (that will include the main paper) and then see how long it takes to get and parse the pdfs
for i in similarityScoresWithTitle[:10]:
    listOfSections = get_text(get_paperId_from_query(i[1]))
    print(f"listOfSections is {listOfSections}")
    if(listOfSections):
        embeddings[i[1]] = []
        for index, value in enumerate(listOfSections):
            print(f"{type(i[1])} and {type(index)} and {type(listOfSections)} and {type(value)}")
            embeddings[i[1]].append((model.encode(value), index, value, i[1]))
print(f"Run time = {time.perf_counter() - start}")

In [ ]:
# for index, words in enumerate(mainPaperSections):
#     print(f"Index {index}:\n{words}")

sectionSimilarityScores is a list of lists where each sublist is a similarity score of the section embeddings and then
each sublist is -> Section Embedding, Index into the section, the section text, the title again(?)

In [ ]:
# Compare a section against all the others and return the best 5
# TODO Create a general variable for the main paper
# Compare the 7th section to all the other sections
sectionSimilarityScores = []

for key, value in embeddings.items():
    if(key == "BERT Rediscovers the Classical NLP Pipeline"):
        continue
    for i in range(len(value)):
        # print(value[i])
        sectionSimilarityScores.append([newCosineSimilarity(torch.tensor(embeddings["BERT Rediscovers the Classical NLP Pipeline"][7][0]),  torch.tensor(value[i][0])), value[i]])

In [ ]:
# A function to return just the first value of the list to be used in list.sort()
def getKeySecondSort(listOfCosAndValue):
    return listOfCosAndValue[0]

In [ ]:
# Sort the sections 
sectionSimilarityScores.sort(reverse=True, key=getKeySecondSort)

In [ ]:
# print(sectionSimilarityScores[1][1][2])
for i in sectionSimilarityScores:
    print(f"Score: {i[0]} and text: {i[1][2]}")